In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import pandas as pd
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units
import glob
%matplotlib qt

In [2]:
file_path = '/archive/Raphael.Dussin/xanadu_esm4_20190304_mom6_2019.08.08/OM4p125_JRA55do1.4_cycle1/gfdl.ncrc4-intel16-prod/pp/ocean_monthly_z_d2/';
file_name = glob.glob(file_path+'av/monthly_1yr/*.nc')
topo_file = 'ocean_monthly_z_d2.static.nc';
print(file_name[0])

/archive/Raphael.Dussin/xanadu_esm4_20190304_mom6_2019.08.08/OM4p125_JRA55do1.4_cycle1/gfdl.ncrc4-intel16-prod/pp/ocean_monthly_z_d2/av/monthly_1yr/ocean_monthly_z_d2.2018.01.nc


In [3]:
fil = xr.open_dataset(file_name[6])
data =  fil.metpy.parse_cf()

fil1 = xr.open_dataset(file_path+topo_file)
data1 =  fil1.metpy.parse_cf()

Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_longitude for projection grid_mapping variable
Found lat/lon values, assuming latitude_lon

In [4]:
print(data)
print(data.xh)
print(data.yh)
print(data1)

<xarray.Dataset>
Dimensions:     (nv: 2, time: 1, xh: 1440, xq: 1440, yh: 1120, yq: 1120, z_i: 36, z_l: 35)
Coordinates:
  * xh          (xh) float64 -298.6 -298.4 -298.1 -297.9 ... 60.7 60.93 61.16
  * yh          (yh) float64 -83.62 -83.54 -83.46 -83.39 ... 89.7 89.81 89.92
  * z_l         (z_l) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * time        (time) object 2018-07-16 12:00:00
    crs         object Projection: latitude_longitude
  * z_i         (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
  * xq          (xq) float64 -298.5 -298.3 -298.1 -297.8 ... 60.77 61.0 61.23
  * yq          (yq) float64 -83.6 -83.52 -83.45 -83.37 ... 89.73 89.84 89.95
  * nv          (nv) float64 1.0 2.0
Data variables:
    volcello    (time, z_l, yh, xh) float32 ...
    thetao      (time, z_l, yh, xh) float32 ...
    so          (time, z_l, yh, xh) float32 ...
    obvfsq      (time, z_i, yh, xh) float32 ...
    uo          (time, z_l, yh, xq) float32 ...
    vo    

In [5]:
# Specify domain lat/lon limits F1 and S1
lat_lim1 = np.array([-50, -60]);
lat_lim2 = np.array([-40, -50]);
lon_lim1 = np.array([-270, -185]);
lon_lim2 = np.array([-255, -170]);

In [10]:
# Compute gradietns using metpy
x, y = data1['deptho'].metpy.coordinates('x', 'y');
depth = data1['deptho'];
# lat, lon = xr.broadcast(y, x);
lat = data1['geolat']
lon = data1['geolon']
f = mpcalc.coriolis_parameter(lat)
depth_crs = data1['deptho'].metpy.cartopy_crs;
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat, initstring=depth_crs.proj4_init)
#dx = data1['dxt']
#dy = data1['dyt']
depth_y, depth_x = mpcalc.gradient(depth, deltas=(dy, dx))
beta, tmp = mpcalc.gradient(f, deltas = (dy, dx))

In [15]:
print(data.obvfsq)
#beta, tmp = mpcalc.gradient(f, deltas = (dy, dx))
#plt.contourf(x, y, beta)
#plt.colorbar()

<xarray.DataArray 'obvfsq' (time: 1, z_i: 36, yh: 1120, xh: 1440)>
[58060800 values with dtype=float32]
Coordinates:
  * xh       (xh) float64 -298.6 -298.4 -298.1 -297.9 ... 60.46 60.7 60.93 61.16
  * yh       (yh) float64 -83.62 -83.54 -83.46 -83.39 ... 89.59 89.7 89.81 89.92
  * time     (time) object 2018-01-16 12:00:00
    crs      object Projection: latitude_longitude
  * z_i      (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
Attributes:
    long_name:      Square of seawater buoyancy frequency
    units:          s-2
    standard_name:  square_of_brunt_vaisala_frequency_in_sea_water
    cell_methods:   area:mean z_i:point yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT


In [20]:
%matplotlib qt
extent = [lon_lim1[0], lon_lim2[0], lat_lim1[0], lat_lim2[0]];
fig = plt.figure()
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude = data.xh[720] + 150))#central_longitude = data.xh[720] + 120))
#ax.set_extent(extent)
ax.gridlines(linewidth = 0.5)
#ax.coastlines(linewidth = 0.8)
ax.add_feature(cfeature.LAND, edgecolor='black')
#feature = ax.add_feature(coast, edgecolor='gray')
#f = ax.contourf(data.xh, data.yh, data.thetao.metpy.loc[{'time': data.time[0], 'vertical': data.z_l[0]}], 
#             20, transform=ccrs.PlateCarree(), cmap=cm.get_cmap("Spectral"), cbar_kwargs={'shrink': 0.4})
data.thetao.metpy.loc[{'time': data.time[0], 'vertical': data.z_l[0]}].plot(ax = ax, 
       transform=ccrs.PlateCarree(), vmin = 0, vmax = 30, cmap=cm.get_cmap("Spectral"), cbar_kwargs={'shrink': 0.4})
#plt.colorbar(f, ax=ax)
ax.set_ylabel("lat")
ax.set_xlabel("lon")

Text(0.5, 0, 'lon')

In [14]:
%matplotlib qt
fig, axs = plt.subplots(nrows = 4, ncols = 1, sharex = True)
pcm = axs[0].contourf(data.xh, data.yh, data.thetao.metpy.loc[{'time': data.time[0], 'vertical': data.z_l[0]}], 
                      np.arange(-2, 22, 2), extend = 'both', cmap = cm.viridis)
fig.colorbar(pcm, ax = axs[0])
axs[0].set_ylabel('lat')
axs[0].set_ylim(-80, -30)
axs[0].set_title('Surface Potential Temperature (DegC)')

pcm = axs[1].contourf(data.xq, data.yh, data.uo.metpy.loc[{'time': data.time[0], 'vertical': data.z_l[0]}], 
                      np.arange(-0.5, 0.6, 0.05), extend = 'both', cmap = cm.Spectral_r)
fig.colorbar(pcm, ax=axs[1])
axs[1].set_ylabel('lat')
axs[1].set_title('Surface Zonal Velocity')
axs[1].set_ylim(-80, -30)

pcm = axs[2].contourf(data.xh, data.yq, data.vo.metpy.loc[{'time': data.time[0], 'vertical': data.z_l[0]}],
                      np.arange(-0.5, 0.6, 0.05), extend = 'both', cmap = cm.Spectral_r)
fig.colorbar(pcm, ax = axs[2])
axs[2].set_ylabel('lat')
axs[2].set_title('Surface Meridional Velocity')
axs[2].set_ylim(-80, -30)

pcm = axs[3].contourf(x,y,np.sqrt(f**2*depth_x**2 + f**2*depth_y**2)/(beta*4000), np.arange(0, 51, 1), 
                      extend = 'both', cmap = cm.nipy_spectral)
fig.colorbar(pcm,ax=axs[3])
axs[3].set_ylabel('lat')
axs[3].set_xlabel('lon')
axs[3].set_title('f*grad(eta_b)/(beta*H)')
axs[3].set_ylim(-80, -30)

(-80, -30)

In [20]:
plt.contourf(data.xh, data.yh, data.obvfsq.metpy.loc[{'time': data.time[0], 'vertical': data.z_i[5]}], 20,
                      extend = 'both', cmap = cm.Spectral)
plt.colorbar()

In [7]:
fig.savefig('/home/Hemant.Khatri/Work/U_T_Southern_07_2018.png', format = 'png', dpi=150)

In [8]:
data.close()
data1.close()